Lab 5
var 5
Model training and testing.

Библиотеки

In [ ]:
import torch
import os
import pandas as pd
import cv2
import glob
from sklearn.model_selection import train_test_split
import numpy as np
import matplotlib.pyplot as plt
from torchvision import transforms
from PIL import Image 

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
torch.manual_seed(1234)
if device =='cuda':
torch.cuda.manual_seed_all(1234)
print(device)

Разделение набора данных на выборки

In [ ]:
# Define paths
val_path = os.path.abspath('val_list')
test_path = os.path.abspath('test_list')
train_path = os.path.abspath('train_list')

# Read and preprocess the CSV file
df = pd.read_csv('annotations_3.csv', sep=',', header=None)
df = df.drop(df.index[0])
df.drop(0, axis=1, inplace=True)
df.rename(columns={1: 'Absolute_path', 2: 'Class'}, inplace=True)
df.reset_index(inplace=True)
print(df)

# Create directories if they do not exist
for path in [val_path, test_path, train_path]:
if not os.path.isdir(path):
os.mkdir(path)

# Function to load and save images
def load_image(df, path, i):
image_path = os.path.abspath(os.path.join(*df.Absolute_path[i].split("\\")))
image = cv2.imread(image_path)
cv2.imwrite(os.path.join(path, f'{i}.jpg'), image)

# Process images based on ranges
for i in range(900, 1000):
load_image(df, val_path, i)
for i in range(1900, 1999):
load_image(df, val_path, i)
for i in range(800, 900):
load_image(df, test_path, i)
for i in range(1800, 1900):
load_image(df, test_path, i)
for i in range(800):
load_image(df, train_path, i)

# Create lists of image files and split the training list
train_list = glob.glob(os.path.join(train_path, '*.jpg'))
test_list = glob.glob(os.path.join(test_path, '*.jpg'))

train_list, val_list = train_test_split(train_list, test_size=0.1)

Проверка

In [ ]:
random_idx = np.random.choice(200, size=10, replace=False)

fig, axes = plt.subplots(2, 5)
for i, ax in enumerate(axes.flat):
img = cv2.imread(train_list[random_idx[i]])
ax.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))

plt.show()

Увелечение изображений

In [ ]:
common_transforms = [
transforms.Resize((224, 224)),
transforms.RandomResizedCrop(224),
transforms.RandomHorizontalFlip(),
transforms.ToTensor()
]

train_transforms = transforms.Compose(common_transforms)
val_transforms = transforms.Compose(common_transforms)
test_transforms = transforms.Compose(common_transforms)

Загрузка 

In [ ]:
class CustomDataset(torch.utils.data.Dataset):
def __init__(self, file_list, transform=None):
self.file_list = file_list
self.transform = transform

def __len__(self):
return len(self.file_list)

def __getitem__(self, idx):
img_path = self.file_list[idx]
img = Image.open(img_path)

if self.transform:
img_transformed = self.transform(img)
else:
img_transformed = img
label = 1 if 'brown bear' in img_path else 0 # Assuming file names contain 'dog' or 'cat'

return img_transformed, label

In [ ]:
train_data = CustomDataset(train_list, transform=train_transforms)
test_data = CustomDataset(test_list, transform=test_transforms)
val_data = CustomDataset(val_list, transform=val_transforms)

Проверка

In [ ]:
print(train_data[1])
print(val_data[0][1])